In [1]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import torch as pt
from models.common import PerceptLossNet
from loader.dataset import ImageLoader, image2patches
%matplotlib inline

In [ ]:
# Path to reference and distorted iamges
ref_path = './example_images/ldr/i06.bmp'
dist_path ='./example_images/ldr/i06_15_5.bmp'
dynamic_range = 'ldr'


# Parameters of the display model (Assuming peak and black level of a display on which LDR image is shown).
# Set to 100 and 0.5 if unsure. The parameter is not used for HDR images as these are given in luminance values.
top_l = 100
bot_l = 0.5

# The quality assessment model operates on 64x64 patches sampled on a regular grid. 
# The shift specifies the window shift for sampling the patchs. The smaller the shift the more accurate the model is.
stride = 16
saved_state_model = '../net_only.pt'
state = pt.load(saved_state_model, map_location='cpu')
loader = ImageLoader()

def read_convert_pt_image(image_path):
    image = imageio.imread(image_path)
    image = pt.from_numpy(imageio.core.asarray(image))
    image = image.permute(2,0,1)
    return image

# Run perceptual loss network (VGG style)

In [ ]:
# Create the network 
net = PerceptLossNet(state)
# Switch off the training mode
net.eval()

# Read images and convert them into the pytorch tensorc with dimensions NCHW
image_ref = read_convert_pt_image(ref_path)
image_dis = read_convert_pt_image(dist_path)
image_ref = image_ref.unsqueeze(0)
image_dis = image_dis.unsqueeze(0)

# Run the network without recorcding the gradient
with pt.no_grad():
    feature1r, feature2r, feature3r, feature4r, feature5r = net(image_ref,im_type=dynamic_range, lum_top=top_l, lum_bottom=bot_l)
    feature1d, feature2d, feature3d, feature4d, feature5d = net(image_dis,im_type=dynamic_range, lum_top=top_l, lum_bottom=bot_l)


